In [1]:
!pip install optuna

In [2]:
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import optuna

In [3]:
wine = load_wine()

In [4]:
np.unique(wine['target'], return_counts=True)

(array([0, 1, 2]), array([59, 71, 48]))

In [5]:
wine['data'].shape

(178, 13)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(wine['data'], wine['target'], 
                                                    random_state = 42,
                                                    stratify = wine['target'], 
                                                    test_size = 0.3, 
                                                    shuffle = True)

In [7]:
X_train.shape, X_test.shape

((124, 13), (54, 13))

In [8]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [9]:
class WineNet(torch.nn.Module):
    def __init__(self, n_input, n_hidden_neurons):
        super().__init__()
        self.dense1 = torch.nn.Linear(n_input, n_hidden_neurons)
        self.activate = torch.nn.ReLU()
        self.dense2 = torch.nn.Linear(n_hidden_neurons, 3)
        
    def forward(self, x):
        x = self.dense1(x)
        x = self.activate(x)
        x = self.dense2(x)
        return x

In [10]:
def train_model(n_hidden, batch_size, lr, weight_decay, optimize = True):

    wine_net = WineNet(n_input = X_train.shape[1], n_hidden_neurons = n_hidden)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(wine_net.parameters(), lr = lr, weight_decay = weight_decay)

    val_accuracy_hist = []
    train_accuracy_hist = []
    val_loss_hist = []
    train_loss_hist = []

    prev_val_loss = None
    cnt = 0
    for epoch in range(1000):
        order = np.random.permutation(len(X_train))
        for start_index in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_indexes = order[start_index : start_index + batch_size]
            
            X_batch = X_train[batch_indexes, :]
            y_batch = y_train[batch_indexes]

            y_pred = wine_net(X_batch)
            loss_val = loss(y_pred, y_batch)
            loss_val.backward()
            optimizer.step() 

        pred_train = wine_net(X_train)
        pred_val = wine_net(X_test)

        train_loss = loss(pred_train, y_train).detach().numpy()
        val_loss = loss(pred_val, y_test).detach().numpy()

        train_acc = ((pred_train.argmax(dim = 1) == y_train).float().sum() / len(y_train)).detach().numpy()
        val_acc = ((pred_val.argmax(dim = 1) == y_test).float().sum() / len(y_test)).detach().numpy()

        if epoch % 100 == 0:
            print("""Performance:\ntrain_loss {0: .3f}, val_loss {1: .3f}\ntrain_accuracy {2: .2f}, val_accuracy {3: .2f}\n""".format(train_loss, 
                                                                                                                                      val_loss, 
                                                                                                                                      train_acc, 
                                                                                                                                      val_acc)
            )

        train_loss_hist.append(train_loss)
        val_loss_hist.append(val_loss)
        train_accuracy_hist.append(train_acc)
        val_accuracy_hist.append(val_acc)

        if prev_val_loss is not None and val_loss > prev_val_loss and cnt == 5:
            break
        elif prev_val_loss is not None and val_loss > prev_val_loss and cnt < 5:
          cnt += 1
        elif prev_val_loss is not None and val_loss < prev_val_loss:
          cnt = 0
        prev_val_loss = val_loss

    if optimize:
      preds = wine_net(X_test)
      return ((preds.argmax(dim = 1) == y_test).float().sum() / len(y_test)).detach().numpy()

    else:
      return wine_net, train_loss_hist, val_loss_hist, train_accuracy_hist, val_accuracy_hist

In [11]:
def optimize_model(trial):
  lr = trial.suggest_loguniform('learning rate', 1e-4, 1e-2)
  n_hidden = trial.suggest_int('n_hidden', X_train.shape[1] // 2, round(X_train.shape[1] * 2/3), 1)
  batch_size = trial.suggest_int('batch_size', 1, 32, 31)
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)
  return train_model(n_hidden, batch_size, lr, weight_decay, optimize = True)

In [ ]:
study = optuna.create_study(direction = "maximize")
study.optimize(optimize_model, n_trials = 100)

[I 2021-01-24 10:50:25,903] A new study created in memory with name: no-name-0b1ef2cd-f9ca-45f4-8baa-5b2b4f3b0b89


Performance:
train_loss  3.456, val_loss  3.709
train_accuracy  0.52, val_accuracy  0.50

Performance:
train_loss  0.394, val_loss  0.405
train_accuracy  0.84, val_accuracy  0.78

Performance:
train_loss  0.187, val_loss  0.177
train_accuracy  0.94, val_accuracy  0.94

Performance:
train_loss  0.167, val_loss  0.185
train_accuracy  0.95, val_accuracy  0.89

Performance:
train_loss  0.116, val_loss  0.101
train_accuracy  0.94, val_accuracy  0.98

Performance:
train_loss  0.157, val_loss  0.190
train_accuracy  0.96, val_accuracy  0.93

Performance:
train_loss  0.118, val_loss  0.113
train_accuracy  0.96, val_accuracy  0.98

Performance:
train_loss  0.920, val_loss  0.745
train_accuracy  0.65, val_accuracy  0.69

Performance:
train_loss  0.175, val_loss  0.214
train_accuracy  0.93, val_accuracy  0.91

Performance:
train_loss  1.751, val_loss  1.933
train_accuracy  0.66, val_accuracy  0.63



[I 2021-01-24 10:51:50,204] Trial 0 finished with value: 0.6666666865348816 and parameters: {'learning rate': 0.0032443399415620655, 'n_hidden': 8, 'batch_size': 1, 'weight_decay': 0.0025009662145688938}. Best is trial 0 with value: 0.6666666865348816.


Performance:
train_loss  39.935, val_loss  37.946
train_accuracy  0.33, val_accuracy  0.33

Performance:
train_loss  0.274, val_loss  0.281
train_accuracy  0.90, val_accuracy  0.94

Performance:
train_loss  0.171, val_loss  0.185
train_accuracy  0.94, val_accuracy  0.94

Performance:
train_loss  0.140, val_loss  0.160
train_accuracy  0.94, val_accuracy  0.93

Performance:
train_loss  0.120, val_loss  0.140
train_accuracy  0.97, val_accuracy  0.96

Performance:
train_loss  0.091, val_loss  0.108
train_accuracy  0.97, val_accuracy  0.98

Performance:
train_loss  0.080, val_loss  0.093
train_accuracy  0.98, val_accuracy  0.98

Performance:
train_loss  0.078, val_loss  0.090
train_accuracy  0.97, val_accuracy  0.98

Performance:
train_loss  0.066, val_loss  0.084
train_accuracy  0.98, val_accuracy  0.98

Performance:
train_loss  0.060, val_loss  0.096
train_accuracy  0.98, val_accuracy  0.98



[I 2021-01-24 10:51:53,493] Trial 1 finished with value: 0.9814814925193787 and parameters: {'learning rate': 0.003904607162474037, 'n_hidden': 8, 'batch_size': 32, 'weight_decay': 0.0006343378119098824}. Best is trial 1 with value: 0.9814814925193787.


...

Performance:
train_loss  61.628, val_loss  63.137
train_accuracy  0.40, val_accuracy  0.39

Performance:
train_loss  0.354, val_loss  0.377
train_accuracy  0.87, val_accuracy  0.91

Performance:
train_loss  0.230, val_loss  0.261
train_accuracy  0.94, val_accuracy  0.91

Performance:
train_loss  0.171, val_loss  0.174
train_accuracy  0.94, val_accuracy  0.94

Performance:
train_loss  0.158, val_loss  0.164
train_accuracy  0.94, val_accuracy  0.94

Performance:
train_loss  0.135, val_loss  0.138
train_accuracy  0.95, val_accuracy  0.96

Performance:
train_loss  0.122, val_loss  0.127
train_accuracy  0.94, val_accuracy  0.98

Performance:
train_loss  0.119, val_loss  0.125
train_accuracy  0.98, val_accuracy  0.96

Performance:
train_loss  0.121, val_loss  0.136
train_accuracy  0.94, val_accuracy  0.93

Performance:
train_loss  0.097, val_loss  0.100
train_accuracy  0.96, val_accuracy  0.98



[I 2021-01-24 11:49:52,422] Trial 71 finished with value: 0.9814814925193787 and parameters: {'learning rate': 0.0001491979427970002, 'n_hidden': 7, 'batch_size': 1, 'weight_decay': 0.00036833697884633394}. Best is trial 1 with value: 0.9814814925193787.


Performance:
train_loss  16.360, val_loss  17.449
train_accuracy  0.65, val_accuracy  0.57

Performance:
train_loss  0.201, val_loss  0.235
train_accuracy  0.94, val_accuracy  0.89

Performance:
train_loss  0.101, val_loss  0.145
train_accuracy  0.95, val_accuracy  0.94

Performance:
train_loss  0.098, val_loss  0.157
train_accuracy  0.98, val_accuracy  0.98

Performance:
train_loss  0.080, val_loss  0.174
train_accuracy  0.96, val_accuracy  0.94

Performance:
train_loss  0.088, val_loss  0.131
train_accuracy  0.95, val_accuracy  0.94

Performance:
train_loss  0.129, val_loss  0.138
train_accuracy  0.96, val_accuracy  0.94

Performance:
train_loss  0.187, val_loss  0.336
train_accuracy  0.92, val_accuracy  0.83

Performance:
train_loss  0.055, val_loss  0.161
train_accuracy  0.98, val_accuracy  0.94

Performance:
train_loss  0.061, val_loss  0.083
train_accuracy  0.98, val_accuracy  0.98



[I 2021-01-24 11:51:19,401] Trial 72 finished with value: 0.9629629850387573 and parameters: {'learning rate': 0.001006773225142658, 'n_hidden': 7, 'batch_size': 1, 'weight_decay': 0.00021568703216259854}. Best is trial 1 with value: 0.9814814925193787.


Performance:
train_loss  16.238, val_loss  14.693
train_accuracy  0.10, val_accuracy  0.09

Performance:
train_loss  0.497, val_loss  0.519
train_accuracy  0.76, val_accuracy  0.70

Performance:
train_loss  0.164, val_loss  0.155
train_accuracy  0.94, val_accuracy  1.00

Performance:
train_loss  0.164, val_loss  0.164
train_accuracy  0.92, val_accuracy  0.91

Performance:
train_loss  0.097, val_loss  0.097
train_accuracy  0.95, val_accuracy  0.98

Performance:
train_loss  0.104, val_loss  0.104
train_accuracy  0.94, val_accuracy  0.94

Performance:
train_loss  0.080, val_loss  0.128
train_accuracy  0.96, val_accuracy  0.94

Performance:
train_loss  0.072, val_loss  0.108
train_accuracy  0.98, val_accuracy  0.98

Performance:
train_loss  0.133, val_loss  0.160
train_accuracy  0.93, val_accuracy  0.93

Performance:
train_loss  0.060, val_loss  0.089
train_accuracy  0.98, val_accuracy  0.98



In [ ]:
params = study.best_params

In [ ]:
model, train_loss_hist, val_loss_hist, train_accuracy_hist, val_accuracy_hist = train_model(params['n_hidden'], 
                                                                                            params['batch_size'], 
                                                                                            params['learning rate'], 
                                                                                            params['weight_decay'], 
                                                                                            optimize = False)

In [ ]:
plt.rcParams["figure.figsize"] = (10, 5)
plt.plot(train_loss_hist)
plt.plot(val_loss_hist)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)
plt.plot(train_loss_hist[-400:])
plt.plot(val_loss_hist[-400:])
plt.show()

In [ ]:
plt.plot(train_accuracy_hist)
plt.plot(val_accuracy_hist)
plt.show()